# Lab 3 - QMSS 5015
## Cindy Chen, cjc2279

__*I will use the same data set that I used in Lab 2, which is the 2020 HMDA US individual mortgage applications where each row is a unique application. It incorporates 99 variables, documenting demographic and financial information about the applicant, mortgage details, as well as approval/denial outcomes.*__

In [2]:
#load all the packages I need for this lab
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as mpl
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [3]:
#my dataset is 9 GB so I am only bringing in 8,000 rows... otherwise I don't have enough RAM
os.chdir("C:/Users/Cindy C/Downloads/2020_lar/")
mortgage = pd.read_csv("2020_lar.txt", sep = "|", header = 0, nrows=8000)

#preview of the first 10 rows of the data set
mortgage.head(10)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2020,549300HTRLFU2IIL4380,12940,LA,22037.0,2.203795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,7086,53.82,73800,81.0,1589,2010,28
1,2020,549300HTRLFU2IIL4380,99999,TX,48467.0,4.846795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7182,11.04,59100,120.0,2180,3048,29
2,2020,549300HTRLFU2IIL4380,99999,TX,48337.0,4.833795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4654,12.01,59100,114.0,1753,2709,32
3,2020,549300HTRLFU2IIL4380,99999,TX,48379.0,4.837995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6083,9.58,59100,113.0,2076,2995,23
4,2020,549300HTRLFU2IIL4380,26420,TX,48291.0,4.829170e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5394,17.95,80000,89.0,1798,2627,25
5,2020,549300HTRLFU2IIL4380,99999,TX,48249.0,4.824995e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,7209,79.11,59100,128.0,1589,2393,44
6,2020,549300HTRLFU2IIL4380,99999,TX,48363.0,4.836300e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2897,16.26,59100,117.0,857,2832,34
7,2020,549300HTRLFU2IIL4380,99999,OK,40001.0,4.000138e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4584,59.79,57500,88.0,1351,1945,34
8,2020,549300HTRLFU2IIL4380,99999,LA,22027.0,2.202795e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,3.0,NaN,NaN,2741,57.57,50400,94.0,599,1334,49
9,2020,549300HTRLFU2IIL4380,26420,TX,48201.0,4.820133e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,3.0,NaN,NaN,11288,92.36,80000,92.0,2662,3192,11


### __1. Run a simple bivariate regression, and interpret your results.  (Did the results fit your expectations?  Why?  Why not?)__

I ran a simple bivariate regression on 'loan_amount' and 'income'.  My hypothesis is that there is a relationship between income and loan_amount: the higher an applicant's income, the greater loan they are willing to apply for.

First, I transformed income using the log function to create a more linear relationship since income tends to be an logarithmic attribute.  When I view the descriptive statistics for income, I can see that it ranges from -\\$4,000 to \\$8.8 million, with the mean at \$107,550. This wide range suggests that it might be logarithmic, so I will transform the independent variable.

When I ran my bi-variate regression, I can see that my P-value is very close to 0.000 so the relationship is statistically significant if I set my threshold to 0.05.  My R-squared value is 0.245, which is a big improvement from my R-squared of 0.078 when I ran a bivariate regression between the untransformed 'income' variable and 'loan_amount'.  While the negative intercept doesn't make sense, I understand that this is a matter of my regression's extrapolation to very small values that would not exist in reality (it is highly unlikely that income is negative and that requested loans would be close to \\$0).  The coefficient of \\$102,900 seems surprisingly steep; I can interpret the coefficient to mean that for every percent increase in income, an applicant will increase their loan amount by \\$102,900.

Beyond that, the results fit my general expectations, because I believed that a relationship exists between the two variables though I did not think they were tightly correlated.  The R-squared seems within my expectations even though I think the coefficient seems high.

In [4]:
mortgage['income'].describe()

count    7738.000000
mean      107.550788
std       141.720717
min        -4.000000
25%        55.000000
50%        84.000000
75%       130.000000
max      8800.000000
Name: income, dtype: float64

In [5]:
#Transform income using log function
mortgage['income_ln'] = np.where(mortgage['income'] != 0, np.log(mortgage['income']), 0)

#check that the new column prints as expected
#in this preview, we can see that NaN values remain NaN
mortgage[['income', 'income_ln']][390:400]

C:\Users\Cindy C\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Cindy C\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,income,income_ln
390,88.0,4.477337
391,143.0,4.962845
392,NaN,NaN
393,89.0,4.488636
394,41.0,3.713572
395,113.0,4.727388
396,NaN,NaN
397,66.0,4.189655
398,74.0,4.304065
399,66.0,4.189655


In [17]:
#this is the transformed income variable regression
bivar_regression = smf.ols('loan_amount ~ income_ln', data = mortgage).fit(cor_type = "HC3")
bivar_regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            loan_amount   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     2511.
Date:                Sat, 23 Oct 2021   Prob (F-statistic):               0.00
Time:                        13:51:02   Log-Likelihood:            -1.0160e+05
No. Observations:                7737   AIC:                         2.032e+05
Df Residuals:                    7735   BIC:                         2.032e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.327e+05   9225.253    -25.223      0.000   -2.51e+05   -2.15e+05
income_ln   1.029e+05   2054.185     50.106      0.000    9.89e+04    1.07e+05
==============================================================================
Omnibus:                     1058.626   Durbin-Watson:                   1.287
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3187.826
Skew:                           0.720   Prob(JB):                         0.00
Kurtosis:                       5.795   Cond. No.                         31.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#to illustrate the difference in transforming the income, I have included the regression summary
#for the untransformed income variable
bivar_regression = smf.ols('loan_amount ~ income', data = mortgage).fit(cor_type = "HC3")
bivar_regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            loan_amount   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     658.1
Date:                Sat, 23 Oct 2021   Prob (F-statistic):          2.26e-139
Time:                        13:51:02   Log-Likelihood:            -1.0239e+05
No. Observations:                7738   AIC:                         2.048e+05
Df Residuals:                    7736   BIC:                         2.048e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.944e+05   1926.645    100.890      0.000    1.91e+05    1.98e+05
income       277.8270     10.830     25.654      0.000     256.598     299.056
==============================================================================
Omnibus:                     1869.044   Durbin-Watson:                   1.226
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           119320.573
Skew:                           0.058   Prob(JB):                         0.00
Kurtosis:                      22.237   Cond. No.                         223.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### __2. Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results.  Did it work out?  Yes?  No?__

I've added the variable "combined_loan_to_value_ratio" to partially explain the association that I see.  Since this variable incorporates the downpayment that the applicant is willing to offer (because a larger downpayment would reduce the loan size compared to the value of the home), my hypothesis is that it will better explain the relationship between the mortgage size requested and the income of the applicant.  Since I thought the ln(income) coefficient seemed too big, it's likely because the downpayment size is related to applicants' incomes.  After all, those with high incomes can save money for a downpayment more easily, allowing them to take on an even larger loan to buy their 'dream home'.  In running a multivariate regression, I can isolate the effects of income and a proxy for downpayment.

In preparing to run my new regression, I transformed the "combined_loan_to_value_ratio" using the natural log.  When I ran this new regression, my P-values for both variables remained significant and my R-squared value improved from 0.245 to 0.371, a material improvement in my model's ability to explain the data (where much less can be attributed to randomness).

In terms of the equation derived from this new regression, it fits my hypothesis about better explaining the relationship between income and loan amount.  The summary tells me that for every 1% increase in income, the requested loan increases by \\$99,830 (this is a smaller coefficient than before).  Meanwhile, every 1% increase in the loan-to-value ratio (which factors in the downpayment) leads to a \$94,940 increase in the requested loan.  This second description makes sense: if a $95K jump in loans results in a mere 1% increase in my loan-to-value ratio, it suggests that my downpayment must be quite hefty to keep my change in ratio so small.  With such a large downpayment as it relates to home value, an applicant is more comfortable requesting a larger mortgage.

In [19]:
#turn the combined loan to value ratio from an object to a numeric variable
mortgage['combined_loan_to_value_ratio'] = pd.to_numeric(mortgage['combined_loan_to_value_ratio'],
                                                         errors = 'coerce')

#view descriptive statistics for this variable so I can decide if we need to scale or transform it.
mortgage['combined_loan_to_value_ratio'].describe()

#Transform this ratio using log function
mortgage['cltv_ln'] = np.where(mortgage['combined_loan_to_value_ratio'] != 0,
                               np.log(mortgage['combined_loan_to_value_ratio']), 0)

#check that the new column prints as expected
#in this preview, we can see that NaN values remain NaN
mortgage[['combined_loan_to_value_ratio', 'cltv_ln']][390:400]

,combined_loan_to_value_ratio,cltv_ln
390,NaN,NaN
391,95.000,4.553877
392,NaN,NaN
393,86.500,4.460144
394,70.000,4.248495
395,64.581,4.167920
396,83.315,4.422629
397,100.866,4.613793
398,94.988,4.553751
399,101.384,4.618915


In [20]:
#run regression
new_regression = smf.ols('loan_amount ~ income_ln + cltv_ln',
                           data = mortgage).fit(cor_type = "HC3")
new_regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            loan_amount   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     1715.
Date:                Sat, 23 Oct 2021   Prob (F-statistic):               0.00
Time:                        13:51:02   Log-Likelihood:                -75437.
No. Observations:                5817   AIC:                         1.509e+05
Df Residuals:                    5814   BIC:                         1.509e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6.257e+05   1.52e+04    -41.240      0.000   -6.55e+05   -5.96e+05
income_ln   9.983e+04   2120.928     47.071      0.000    9.57e+04    1.04e+05
cltv_ln     9.494e+04   2870.454     33.074      0.000    8.93e+04    1.01e+05
==============================================================================
Omnibus:                      855.493   Durbin-Watson:                   1.368
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3132.042
Skew:                           0.711   Prob(JB):                         0.00
Kurtosis:                       6.301   Cond. No.                         70.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### __3. Run another multiple regression.  Tell me how you expect your dependent variable to be affected by the independent variables.  Interpret your results.__

This time, I will run a regression on property values.  My dependent variable is property value and my independent variables are the % of minority population in the specific census tract where the home resides along with the median age of the housing units in that census tract. My hypothesis is that areas with higher minorities and older homes tend to observe declining property values, because minorities tend to have lower incomes on average and therefore, their neighborhoods are less 'desirable'.  Likewise, if an area is underinvested because housing developers don't see an area is 'trendy', there will be few new developments so homes will tend to be older, which correlates with lower property values.

In running my regression, my overall expectations were incorrect.  The \% of minorities in a census tract was not a significant variable since the P-value is 0.833.  I also observe a very low R-squared value of 0.002. Meanwhile, the median age of housing units is significant (P-value of 0.001) and its relationship to property value seems intuitive.  Its coefficient of -497.6 means that if the median housing unit is older by a year, property value drops by \\$498.  This small coefficient makes sense since I would not expect a property to decline drastically in value only because the median home is older by a year, especially given US housing demand in 2020.

In [21]:
#convert property value to numeric
mortgage['property_value'] = pd.to_numeric(mortgage['property_value'], errors = 'coerce')

alternate_regression = smf.ols(
    'property_value ~ tract_minority_population_percent + tract_median_age_of_housing_units',
    data = mortgage).fit(cor_type = "HC3")

alternate_regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_value   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     7.344
Date:                Sat, 23 Oct 2021   Prob (F-statistic):           0.000652
Time:                        13:51:02   Log-Likelihood:                -80410.
No. Observations:                5977   AIC:                         1.608e+05
Df Residuals:                    5974   BIC:                         1.608e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          3.156e+05   3500.119     90.183      0.000    3.09e+05    3.23e+05
tract_minority_population_percent    25.1539    119.330      0.211      0.833    -208.776     259.084
tract_median_age_of_housing_units  -497.6078    146.097     -3.406      0.001    -784.011    -211.204
==============================================================================
Omnibus:                     2656.482   Durbin-Watson:                   1.454
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33514.865
Skew:                           1.785   Prob(JB):                         0.00
Kurtosis:                      14.038   Cond. No.                         60.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### __4. Now add another independent variable to that model in Question 3, preferably a set of dummy variables.  Tell me why you added that new set of variables and what effect you expected them to have.  Did they have an effect?  Interpret that new model.__

In this new regression, I have added a new independent variable (occupancy_type) as a set of dummy variables.  I added this new variable because I hypothesize that occupancy_type is related to property value.

According to the HDMA Documentation, the three occupancy codes signal the following residence types:
* Occupancy type 1 = Principal residence
* Occupancy type 2 = Secondary residence
* Occupancy type 3 = investment property

I suspect that if a property is to be used as a primary residence, it will yield higher property values, because this is where the buyers want to spend most of their time.  Meanwhile, a property that's designated as a secondary residence is likely a vacation home or condo, which means they might be in more remote areas or smaller than a principal residence, so it will be less valuable.  Finally, if a property is being used as an investment property, the owners are likely renting it out and hold much less discretion over the quality of the area or housing structure so long as it generates rental income.  In other words, I expect occupancy_type = 1 to increase property value, type = 2 to have a muted effect on property value, and type = 3 to potentially reduce property value.

__Findings__:


When I ran the new regression with 'occupancy_type' dummy variables, it did not significantly change my R-squared value (I will investigate in the next question whether or not the improvement is nonetheless significant). However, occ_1 and occ_2 (which denotes primary vs secondary residence) was statistically significant (but occ_3 for investment properties was not). In all, my hypothesis was incorrect in how these two occupancy types might interact with property value.

If the home will be used as a primary residence, it will add \\$107,000 to the propery value.  However, the most surprising finding was that if a home will be used as a secondary residence, it will add \\$123,800 to the property value (much higher than a primary residence!). Since this mortgage data comes spans all of 2020, housing shifts due to the pandemic might explain why we see a much higher jump in property values for secondary residences.  Since people were limited in where they could travel and move about due to the pandemic, acquiring a secondary residence perhaps in the countryside or in another part of the country grew in demand, leading to higher property values.

In [22]:
new_df = pd.concat([mortgage.property_value,
                    mortgage.tract_minority_population_percent,
                    mortgage.tract_median_age_of_housing_units,
                   pd.get_dummies(mortgage.occupancy_type, prefix = 'occ')], axis = 1)

new_df.head(10)

,property_value,tract_minority_population_percent,tract_median_age_of_housing_units,occ_1,occ_2,occ_3
0,195000.0,53.82,28,1,0,0
1,285000.0,11.04,29,1,0,0
2,265000.0,12.01,32,1,0,0
3,255000.0,9.58,23,1,0,0
4,245000.0,17.95,25,1,0,0
5,145000.0,79.11,44,1,0,0
6,285000.0,16.26,34,1,0,0
7,55000.0,59.79,34,1,0,0
8,195000.0,57.57,49,1,0,0
9,165000.0,92.36,11,1,0,0


In [23]:
occ_type_prop_value = smf.ols(
    'property_value ~ tract_minority_population_percent + tract_median_age_of_housing_units + occ_1 + occ_2 + occ_3',
data = new_df).fit(cor_type = "HC3")

occ_type_prop_value.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         property_value   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     37.62
Date:                Sat, 23 Oct 2021   Prob (F-statistic):           4.00e-31
Time:                        13:51:02   Log-Likelihood:                -80343.
No. Observations:                5977   AIC:                         1.607e+05
Df Residuals:                    5972   BIC:                         1.607e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           2.17e+05   5321.557     40.772      0.000    2.07e+05    2.27e+05
tract_minority_population_percent   -32.7048    118.170     -0.277      0.782    -264.361     198.951
tract_median_age_of_housing_units  -594.8727    144.739     -4.110      0.000    -878.613    -311.132
occ_1                               1.07e+05   5391.299     19.855      0.000    9.65e+04    1.18e+05
occ_2                              1.238e+05   1.32e+04      9.356      0.000    9.79e+04     1.5e+05
occ_3                             -1.389e+04   8776.755     -1.582      0.114   -3.11e+04    3319.908
==============================================================================
Omnibus:                     2707.023   Durbin-Watson:                   1.477
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35976.933
Skew:                           1.812   Prob(JB):                         0.00
Kurtosis:                      14.460   Cond. No.                     3.25e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.02e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### __5. Now run a partial F test comparing the model in Question 3 to the model in Question 4.  Does the F test support the idea of adding those new variables?  Why?  Why not?__

In running my partial F-test between my two models, the partial F-test supports the argument that adding dummy variables for occupancy type improves my model fit.  Since my R-squared value did improve between the models (though I thought the change was immaterial), the partial F-test gives me rich insight that this improvement was quite impactful and that it is statistically different from zero.  We can see this in the F-test results listed below as the probability was 8.216e-30 (much lower than a p-value treshold of 0.05) that this was statistically significant.

In [24]:
sm.stats.anova_lm(alternate_regression, occ_type_prop_value)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,5974.0,1.695823e+14,0.0,NaN,NaN,NaN
1,5972.0,1.658212e+14,2.0,3.761136e+12,67.728104,8.216371e-30
